<a href="https://colab.research.google.com/github/nischalraii/Digits/blob/main/Digits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy
import matplotlib
import scipy

In [ ]:
class neuralNetwork:

  def __init__(self, inputnodes, hiddennodes, secondhiddennodes, outputnodes,learningrate):
    self.inodes = inputnodes
    self.hnodes = hiddennodes
    self.shnodes = secondhiddennodes
    self.onodes = outputnodes
    self.lr = learningrate

    self.wih = numpy.random.normal(0.0, pow(self.hnodes, -0.5),(self.hnodes, self.inodes))
    self.whs = numpy.random.normal(0.0, pow(self.shnodes, -0.5),(self.shnodes, self.hnodes))
    self.wso = numpy.random.normal(0.0,pow(self.onodes, -0.5), (self.onodes, self.shnodes))
    self.activation_function = lambda x: scipy.special.expit(x)
    pass

  # train the neural network
  def train(self, inputs_list, targets_list):
    # convert inputs list to 2d array
    inputs = numpy.array(inputs_list, ndmin=2).T
    targets = numpy.array(targets_list, ndmin=2).T

    # calculate signals into hidden layer
    hidden_inputs = numpy.dot(self.wih, inputs)

    # calculate the signals emerging from hidden layer
    hidden_outputs = self.activation_function(hidden_inputs)

    second_hidden_inputs = numpy.dot(self.whs, hidden_outputs)
    second_hidden_outputs = self.activation_function(second_hidden_inputs)

    # calculate signals into final output layer
    final_inputs = numpy.dot(self.wso, second_hidden_outputs)

    # calculate the signals emerging from final output layer
    final_outputs = self.activation_function(final_inputs)

    output_errors = targets - final_outputs
    second_output_errors = numpy.dot(self.wso.T,output_errors)
    hidden_errors = numpy.dot(self.whs.T, second_output_errors)

    # update the weights for the links between the second hidden and output layers
    self.wso += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(second_hidden_outputs))

    # update the weights for the links between the hidden and output layers
    self.whs += self.lr * numpy.dot((second_output_errors * second_hidden_outputs * (1.0 - second_hidden_outputs)), numpy.transpose(hidden_outputs))

    # update the weights for the links between the input and hidden layers
    self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
    pass


  def query(self,inputs_list):
    inputs = numpy.array(inputs_list,ndmin=2).T

    hidden_inputs = numpy.dot(self.wih,inputs)
    hidden_outputs = self.activation_function(hidden_inputs)

    second_hidden_inputs = numpy.dot(self.whs,hidden_outputs)
    second_hidden_outputs = self.activation_function(second_hidden_inputs)

    final_inputs = numpy.dot(self.wso,second_hidden_outputs)
    final_outputs = self.activation_function(final_inputs)

    return final_outputs


In [ ]:
# number of input, hidden, output nodes and learning rate
input_nodes = 784
hidden_nodes = 100
second_hidden_nodes = 100
output_nodes = 10
learning_rate = 0.05

# create instance of neural network
n = neuralNetwork(input_nodes, hidden_nodes, second_hidden_nodes, output_nodes, learning_rate)


In [ ]:
data_file = open("/content/sample_data/mnist_train_small.csv", 'r')
data_list = data_file.readlines()
data_file.close()

In [ ]:
epoch = 2
for e in range(epoch):
  for record in data_list:
    all_values = record.split(',')
    inputs = numpy.asarray([(float(pixel) / 255.0) * 0.99 + 0.01 for pixel in all_values[1:]])
    targets = numpy.zeros(output_nodes) + 0.01
    targets[int(all_values[0])] = 0.99
    n.train(inputs,targets)
    pass

In [ ]:
test_data_file = open("/content/sample_data/mnist_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()


In [ ]:
#check = numpy.asarray([(float(pixel) / 255.0) * 0.99 + 0.01 for pixel in first_value[1:]])
# len(check)

In [ ]:
scorecard = []
for record in test_data_list:
  # split the record by the ',' commas
  all_values = record.split(',')

  # correct answer is first value
  correct_label = int(all_values[0])

  # scale and shift the inputs
  inputs = numpy.asarray([(float(pixel) / 255.0) * 0.99 + 0.01 for pixel in all_values[1:]])

  # query the network
  outputs = n.query(inputs)

  # the index of the highest value corresponds to the label
  label = numpy.argmax(outputs)

  # append correct or incorrect to list
  if (label == correct_label):
    # network's answer matches correct answer, add 1 to scorecard
    scorecard.append(1)
  else:
    # network's answer doesn't match correct answer, add 0 to scorecard
    scorecard.append(0)
    pass
  pass

# calculate the performance score, the fraction of correct answers
scorecard_array = numpy.asarray(scorecard)
print ("performance = ", scorecard_array.sum() / scorecard_array.size)

performance =  0.9482
